![介绍](img.png)
张量就是多维数组
张量的阶数就是张量的维度
张量的阶数也叫做张量的秩

# Variable
Variable 是pytorch中的变量 在0.4.0版本中Variable已经被弃用 并入到了tensor中

- data：保存Variable所包含的tensor
- grad：保存data对应的梯度，grad也是个Variable而不是tensor，它和data的形状一样
- grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度
  创建这个张量的函数，是自动求导的关键
- requires_grad:指示是否需要梯度
- is_leaf:指示是否是叶子节点，叶子节点指的是用户创建的变量，非叶子节点指的是通过一些操作得到的变量

# Tensor
Tensor是pytorch中的张量，和numpy中的array很类似
Tensor和numpy的array之间最大的区别就是Tensor可以在GPU上加速运算
Tensor和numpy的array之间可以无缝的转换，且二者共享内存

![](img_1.png)
在Variable的基础上，扩展了下列字段

- dtype:指定tensor的数据类型
  - torch.FloatTensor:32位浮点型
  - torch.DoubleTensor:64位浮点型
  - torch.cuda.FloatTensor:32位浮点型，存储在GPU上
- shape:指定张量的形状
  如(64,3,32,32)表示一个64张32x32的彩色图片组成的batch
  (64,3,224,224)表示一个64张224x224的彩色图片组成的batch
- device:当前数据分配到什么内存上，cpu的内存或gpu显存
  - torch.device('cuda')
  - torch.device('cpu')

In [3]:
import torch
import numpy as np
# 创建Tensor
# 从data创建tensor
# 创建一个numpy数组
# arr 是长宽为3的二维数组
arr = np.ones((3,3))

# 从list创建tensor
t1 = torch.tensor(arr, dtype=None, device='cpu', requires_grad=False)
# 对于没有cuda的电脑，下面这句话会报错
# tensor = torch.tensor(arr, dtype=None, device='cuda', requires_grad=False)

print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [6]:
# 从numpy创建tensor 会共享内存
t2 = torch.from_numpy(arr)
print(t2)
print(id(arr))
print(id(t2))


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
140316727855408
140317270392976


In [9]:
# 使用torch.zeros创建tensor
# 会根据指定的shape创建一个全是0的tensor

# 任意声明一个tensor
out_t = torch.tensor([11])
print(out_t)
# create a tensor with shape(3,3) and full of 0
# 并赋值给out_t
# 返回的t指向的内存地址和out_t是一样的
t = torch.zeros((3,3), out=out_t)
print(out_t)
print(t)
print(id(out_t))
print(id(t))
print(id(t) == id(out_t))


# 类似于numpy的使用 还有torch.ones torch.full torch.arange torch.linspace torch.rand torch.normal



tensor([11])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
140317270404896
140317270404896
True


In [10]:
# 使用数来填充的张量
t7 = torch.full((3,3), 7)
print(t7)

tensor([[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]])


In [14]:
# 使用arange创建张量
# 从2到10 前闭后开 步长为2
t8 = torch.arange(2,10,2)
print(t8)

tensor([2, 4, 6, 8])


In [13]:
# 使用linspace创建张量
# 从2到10 前闭后闭 一共有5个数
t9 = torch.linspace(2,10,5)
print(t9)

tensor([ 2.,  4.,  6.,  8., 10.])


In [15]:
# 创建对角张量
# 对角线上的元素为1
# 对角线的偏移量为0
t10 = torch.eye(3,3)
print(t10)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


In [16]:
# 创建正态分布张量 高斯分布
# mean:张量的均值
# std:张量的标准差
t11 = torch.normal(mean=torch.zeros(3,3), std=torch.ones(3,3))
print(t11)

tensor([[-1.0160,  0.4011,  1.5175],
        [-0.8173, -0.7611, -1.3684],
        [ 1.1762,  0.1815, -1.6923]])


In [17]:
# 创建随机张量
t12 = torch.randn(3,3)
print(t12)

tensor([[-1.0130, -1.0850, -0.6784],
        [-0.5729,  0.7610,  0.9752],
        [ 0.8611,  0.4730,  0.5292]])


In [18]:
# 创建均匀分布张量
t13 = torch.rand(3,3)
print(t13)

tensor([[0.9034, 0.1475, 0.3576],
        [0.1560, 0.6016, 0.7550],
        [0.3019, 0.0696, 0.0842]])


In [19]:
# 依概率分布创建张量
# 生成从0到n-1的随机排列
torch.randperm(10)


tensor([0, 4, 5, 7, 2, 3, 9, 1, 8, 6])

In [20]:
# bernoulli分布
t15 = torch.bernoulli(t13)
print(t15)

tensor([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 0.]])


# Tensor的操作


In [31]:
# cat 拼接
# 将多个张量沿着指定的维度拼接在一起，返回一个新的张量。
# 拼接操作会将张量的所有元素沿着指定的维度依次排列，可能会改变张量内部的数据分布。
# tensors：要拼接的张量的列表
# dim：拼接的维度 0表示按行拼接（纵向） 1表示按列拼接（横向）

# 创建 [[1, 2, 3], [4, 5, 6]] 的 tensor
a = torch.tensor([[1, 2, 3], [4, 5, 6]])

b = torch.tensor([[7, 8, 9], [10, 20, 30]])
# 使用 torch.cat() 将 a 和 b 沿着第一个维度拼接
d = torch.cat([a, b], dim=0)
print("torch.cat():")
print(d)
# d的shape
print(d.shape)

torch.cat():
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 20, 30]])
torch.Size([4, 3])


In [32]:
# stack 堆叠
# 将多个张量沿着指定的维度堆叠在一起，返回一个新的张量。
# 堆叠操作会沿着指定的维度将张量一个接一个地放置在一起，不会改变张量内部的数据分布。
# tensors
# dim：拼接的维度
# 使用 torch.stack() 将 a 和 b 沿着第一个维度堆叠
c = torch.stack([a, b], dim=0)
print("torch.stack():")
print(c)
print(c.shape)


torch.stack():
tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 20, 30]]])
torch.Size([2, 2, 3])


In [33]:
print(torch.chunk(c, 2, dim=0))

(tensor([[[1, 2, 3],
         [4, 5, 6]]]), tensor([[[ 7,  8,  9],
         [10, 20, 30]]]))


In [34]:
print(torch.chunk(d, 2, dim=0))

(tensor([[1, 2, 3],
        [4, 5, 6]]), tensor([[ 7,  8,  9],
        [10, 20, 30]]))


In [40]:
# split 分割
print(torch.split(d, 1, dim=0))

(tensor([[1, 2, 3]]), tensor([[4, 5, 6]]), tensor([[7, 8, 9]]), tensor([[10, 20, 30]]))


# 张量的计算
## 加减乘除

## 复合计算 （简洁化）



## 线性回归

